# CONTACT MATRIX TRANSFORMATION

## Preliminaries

In [1]:
import os 
import sys

import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath('../src/')
if module_path not in sys.path:
    sys.path.append(module_path)

import utils as ut
import plots as pt

plt.rcParams.update({'font.size': 15})
plt.rc('axes', labelsize=20)
plt.rcParams['xtick.labelsize'] = 20
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rcParams['pdf.fonttype'] = 42

path_cwd = os.getcwd()
print(path_cwd)

/Users/alfonso/workshop/contact/notebooks


## Update (Time transform)

Original contact matrices from Mistry et al. are 2005 dated. For our computational model we use census data from 2019, so we require a temporal updating. We use the method called 'M2 (density correction)' from Arregui et al [https://doi.org/10.1371/journal.pcbi.1006638]: $M_{ij}'=M_{ij}\frac{N N_j'}{N_j N'}$, where $M_{ij}$ is the original contact matrix, representing the per capita average number of contacts between age group $i$ and $j$, and $M_{ij}'$ is the transformed one, $N_j$ an $N$ are the original age-group $j$ and total population, respectively, and similarly goes for the updated population $N_j'$ and $N'$. 

In [2]:
id_state = 'California'
path_source = os.path.join(path_cwd, '..')

contact_05 = ut.import_contact_matrix(id_state=id_state, path=path_source)

population_05 = ut.import_age_distribution(id_state=id_state, path=path_source)

population_19 = ut.import_age_distribution(id_state=id_state, path=path_source, year=2019, reference=False)

contact_19 = ut.update_contact_matrix(contact_05, population_05, population_19)

Let's check that contact reciprocity is preserved:

In [3]:
ut.check_symmetry_condition(contact_19, population_19)

True

How does the maximum eigenvalue has changed over the years?

In [4]:
max_eigenvalue_05 = np.max(np.linalg.eigvals(contact_05))
max_eigenvalue_19 = np.max(np.linalg.eigvals(contact_19))

print('Maximum eigenvalue after updating. 2005: {0}. 2019: {1}'.format(max_eigenvalue_05, max_eigenvalue_19))

Maximum eigenvalue after updating. 2005: 14.384944251347191. 2019: 13.472919479485212


## Coarse-graining (Age resolution transform)

The spatial resolution of our model is higher than that of the state administrative division. We resolve up until the ZIP code block, but we limit ourselves to some metropolitan areas, and do not cover the full spatial extent of the state subdivision. Thus, we need to adapt the matrices also in space. However, the census data by age at the ZIP level has a lower age resolution than the one we have currently with the state-level population. Therefore, before particularizing the contact matrix into each ZIP code block, we will perform the coarse-graining procedure and reduce the population and contact matrix age resolution from $A=85$ number of age groups to $A'=3$ number of age groups (underage, adults, elders).

In [ ]:
contact_cg19 = ut.coarse_grain_contact_matrix(contact_19, population_19)

population_cg19 = ut.coarse_grain_population(population_19)

As before, let's check for the contact reciprocity condition:

In [ ]:
ut.check_symmetry_condition(contact_cg19, population_cg19)

True

And what about the maximum eigenvalue?

In [ ]:
max_eigenvalue_19 = np.max(np.linalg.eigvals(contact_19))
max_eigenvalue_cg19 = np.max(np.linalg.eigvals(contact_cg19))

print('Maximum eigenvalue after coarse graining. 2019 original: {0}. 2019 coarse-grained: {1}'.format(max_eigenvalue_19, max_eigenvalue_cg19))

Maximum eigenvalue after coarse graining. 2019 original: 14.384944251347196. 2019 coarse-grained: 13.575844144474106


## Region adaptation (Space transform)

Actually, it doesn't matter whether time or space, the transform that has to undergo the contact matrix is the same one employed before for the time updating. The diference lies now on the fact the original matrix and population data corresponds to the 3-age-groups, year 2019, for the state-level, and we now move to 3-age groups, year 2019, for the ZIP-code-block-level. Then, here: $M_{ij}'=M_{ij}\frac{N N_j'}{N_j N'}$, $M_{ij}$, $N_j$, and $N$ correspond to the former setting (3-age-groups year 2019 state-level) and $M_{ij}'$, $N_{j}'$, $N'$ to the last setting (3-age groups, year 2019, ZIP-code-block-level)


#population_patch = load stuff

#contact_patch = ut.update_contact_matrix(contact_cg19, population_cg19, population_patch)